<a href="https://colab.research.google.com/github/ManoharBandam/PSU/blob/master/T5_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd
import numpy as np
import torch


In [2]:
from pathlib import Path

In [3]:
from torch.utils.data import Dataset, DataLoader

In [4]:
!pip install --quiet pytorch-lightning==1.1.3

     |████████████████████████████████| 680 kB 13.0 MB/s 
     |████████████████████████████████| 596 kB 66.8 MB/s 
     |████████████████████████████████| 829 kB 77.4 MB/s 
     |████████████████████████████████| 136 kB 63.2 MB/s 
     |████████████████████████████████| 1.1 MB 79.0 MB/s 
     |████████████████████████████████| 144 kB 63.0 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 271 kB 66.1 MB/s 


In [5]:
!pip install --upgrade pytorch-lightning
import pytorch_lightning as pl

     |████████████████████████████████| 584 kB 12.6 MB/s 
     |████████████████████████████████| 408 kB 70.2 MB/s 
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.1.3
    Uninstalling pytorch-lightning-1.1.3:
      Successfully uninstalled pytorch-lightning-1.1.3


In [6]:
from pytorch_lightning.callbacks import ModelCheckpoint

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
from termcolor import colored

In [9]:
import textwrap

In [10]:
!pip install --quiet transformers==4.1.1
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
)

     |████████████████████████████████| 1.5 MB 12.0 MB/s 
     |████████████████████████████████| 880 kB 72.7 MB/s 
     |████████████████████████████████| 2.9 MB 63.1 MB/s 


In [11]:
pl.seed_everything(42)

Global seed set to 42


42

In [46]:
with Path("sample_data/fetaQA-v1_train.json").open() as json_file:
  data = json.load(json_file)
data_df = pd.json_normalize(df1['data'])
data_df.head()

,feta_id,table_source_json,page_wikipedia_url,table_page_title,table_section_title,table_array,highlighted_cell_ids,question,answer,source
0,12019,totto_source/train_json/example-4318.json,http://en.wikipedia.org/wiki/Core_fonts_for_th...,Core fonts for the Web,List of fonts and files,"[[File name, Font name, Variants, Last version...","[[1, 1], [1, 3], [9, 1], [9, 3], [20, 1], [20,...",What were the font-versions that were availabl...,The font-versions that were available from Cor...,mturk-approved
1,10691,totto_source/train_json/example-2990.json,http://en.wikipedia.org/wiki/Shruti_Haasan_fil...,Shruti Haasan filmography,Filmography,"[[Title, Year, Role(s), Director, Language(s),...","[[1, 0], [1, 1], [1, 4], [1, 5], [2, 0], [2, 1...",In what films has Shruti Haasan made a cameo a...,Shruti Haasan made a cameo appearance in Tamil...,internal
2,8300,totto_source/train_json/example-599.json,http://en.wikipedia.org/wiki/List_of_Assyrian_...,List of Assyrian kings,Dynasty of Puzur-Ashur (2025–1749 BC),"[[#, Image, King, Reign, Succession, Notes, Re...","[[5, 2], [6, 2], [6, 4], [7, 2], [7, 4]]","On the Assyrian King List, for whom is Sargon ...","On the Assyrian King List, Sargon appears as t...",internal
3,8107,totto_source/train_json/example-406.json,http://en.wikipedia.org/wiki/List_of_Stanley_C...,List of Stanley Cup champions,Active teams,"[[Apps, Team, Wins, Losses, Win %, Years of ap...","[[1, 1], [1, 2], [3, 1], [3, 2]]",How many Stanley Cup championships did Toronto...,The Toronto Maple Leafs has won thirteen Stanl...,internal
4,10564,totto_source/train_json/example-2863.json,http://en.wikipedia.org/wiki/Estadio_El%C3%ADa...,Estadio Elías Aguirre,International matches hosted,"[[Date, Team #1, Res., Team #2, Round, Attenda...","[[1, 1], [1, 2], [1, 3], [1, 4], [5, 1], [5, 2...",What were the first and final Copa América mat...,The game that opened the Elías Aguirre's parti...,internal


In [47]:
datas = data['data']

In [15]:
MODEL_NAME = "t5-base"

In [16]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [17]:
sample_encoding = tokenizer(
    "Would I rather be feared or loved?",
    "Easy. Both. I want people to be afraid of how much they love me."
)

In [18]:
sample_encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [19]:
print(sample_encoding["input_ids"])

[5328, 27, 1066, 36, 3, 27625, 42, 1858, 58, 1, 6844, 5, 2867, 5, 27, 241, 151, 12, 36, 7403, 13, 149, 231, 79, 333, 140, 5, 1]


In [20]:
print(sample_encoding["attention_mask"])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [31]:
table = pd.DataFrame(data["table_array"][0])
table.columns = table.iloc[0]
table = table.iloc[1: , :].reset_index(drop=True).astype(str)
item = data.iloc[0]
# display(table)
print(item)
print(item.highlighted_cell_ids)

feta_id                                                             12019
table_source_json               totto_source/train_json/example-4318.json
page_wikipedia_url      http://en.wikipedia.org/wiki/Core_fonts_for_th...
table_page_title                                   Core fonts for the Web
table_section_title                               List of fonts and files
table_array             [[File name, Font name, Variants, Last version...
highlighted_cell_ids    [[1, 1], [1, 3], [9, 1], [9, 3], [20, 1], [20,...
question                What were the font-versions that were availabl...
answer                  The font-versions that were available from Cor...
source                                                     mturk-approved
Name: 0, dtype: object
[[1, 1], [1, 3], [9, 1], [9, 3], [20, 1], [20, 3]]


In [ ]:
# context = []
# for i in range(2):
#   to_proc = datas[i]['table_array']
#   print('to_proc:', to_proc)
  
#   for j in range(len(to_proc)):
#     row_val = to_proc[j]
#     print(j,'th row:',row_val)
#     sample_encoding = tokenizer(row_val)
#     print('input_i:')
#     print(sample_encoding["input_ids"])
#     print('att_mask:')
#     print(sample_encoding["attention_mask"])
  


In [ ]:
context = []
formatted_df = pd.DataFrame()

for i in range(len(datas)):
  temp_table_df = pd.DataFrame()
  table = datas[i]['table_array']
  val_cumulative = ""
  #print('to_proc:', to_proc)
  for j in range(len(table)):
    val = ' '.join(table[j])
    #print(val)
    if j==0:
      val_cumulative = val
    else:
      val_cumulative = val_cumulative + '[SEP]'+ val
  val_cumulative = datas[i]['question']+'[CLS]'+ val_cumulative
  # print(val_cumulative)

  temp_table_df["context"] = val_cumulative
  temp_table_df["answer"] = datas[i]['answer']
  temp_table_df = pd.DataFrame({"context":[val_cumulative],"answer":[datas[i]['answer']]})
  # print(temp_table_df)
  formatted_df = formatted_df.append(temp_table_df, ignore_index=True)



In [59]:
formatted_df.head()

,context,answer
0,What were the font-versions that were availabl...,The font-versions that were available from Cor...
1,In what films has Shruti Haasan made a cameo a...,Shruti Haasan made a cameo appearance in Tamil...
2,"On the Assyrian King List, for whom is Sargon ...","On the Assyrian King List, Sargon appears as t..."
3,How many Stanley Cup championships did Toronto...,The Toronto Maple Leafs has won thirteen Stanl...
4,What were the first and final Copa América mat...,The game that opened the Elías Aguirre's parti...
